In [1]:
import graphlab

In [2]:
matchups = graphlab.SFrame.read_csv("2016_yahoo_matchups_week1-11.csv",delimiter='|')
weeks_elapsed = 11



This non-commercial license of GraphLab Create for academic use is assigned to lukechu1018@gmail.com and will expire on March 30, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1479847797.log


Finished parsing file /Users/lukechu/Desktop/Pycharm/YahooFF2/2016_yahoo_matchups_week1-11.csv

Parsing completed. Parsed 55 lines in 0.079211 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,str,float,float,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/lukechu/Desktop/Pycharm/YahooFF2/2016_yahoo_matchups_week1-11.csv

Parsing completed. Parsed 55 lines in 0.012966 secs.

In [3]:
matchups

Week,TeamOne,ProjectedScoreOne,ScoreOne,Winner,ScoreTwo,ProjectedScoreTwo,TeamTwo
1,The Golden Yogis,107.69,102.2,<,149.46,120.48,Clippernation
1,In TyGod we Trust,109.61,90.46,<,108.44,109.49,Team 8
1,dicksoutforharambe,107.68,96.92,<,112.56,114.6,NomoneyforRedzone
1,Penguins,115.82,106.22,<,134.9,114.46,Romosexual
1,Sit4Anthems,106.49,73.1,<,118.16,98.82,theoutoftowner
2,The Golden Yogis,106.49,84.2,<,110.16,112.87,In TyGod we Trust
2,Clippernation,130.55,111.52,<,123.86,99.84,theoutoftowner
2,dicksoutforharambe,103.34,104.36,>,101.46,110.47,Team 8
2,Penguins,117.62,117.62,>,102.52,111.9,NomoneyforRedzone
2,Sit4Anthems,104.85,117.76,>,92.92,105.34,Romosexual


In [4]:
# sort by score per week
matchup_week_score = {}

for i in range(1, weeks_elapsed+1): 
    matchup_week_score[i] = matchups[matchups["Week"]==i][["TeamOne", "ProjectedScoreOne", "ScoreOne" ]] # alternatively matchups[["Week"], ...].filter_by(1, "Week")
    matchup_week_score[i].rename({"TeamOne":"Team", "ProjectedScoreOne":"ProjectedScore", "ScoreOne":"Score"})
    
    # collision possible, but there are only 16 weeks
    matchup_week_score[i + 20] = matchups[matchups["Week"]==i][["TeamTwo", "ProjectedScoreTwo", "ScoreTwo" ]] # alternatively matchups[["Week"], ...].filter_by(1, "Week")
    matchup_week_score[i + 20].rename({"TeamTwo":"Team", "ProjectedScoreTwo":"ProjectedScore", "ScoreTwo":"Score"})

    matchup_week_score[i] = matchup_week_score[i].append(matchup_week_score[i+20])
    # remember that a column is an SArray (and a row is a dict)
    # alternatively (would be longer) could use .add_row_number, .rename, and reorder column to be last column
    matchup_week_score[i] = matchup_week_score[i].sort(["Score"]).add_column(graphlab.SArray(range(0,matchup_week_score[i].num_rows())), name ="NumberBeaten")

# matchup_week_score

## Expected Wins

In [5]:
# Calculate Expected Wins. EW = sum( # of opponents beaten each week ) / # of opponents

# For each team, group by number beaten
#simple way to initalize new SFrame with same structure and skip an append
matchup_scores = matchup_week_score[1]
for i in range(2, weeks_elapsed + 1):
    matchup_scores = matchup_scores.append(matchup_week_score[i])

# group by sum of total Beaten    
teams_beaten = matchup_scores.groupby(key_columns = "Team", operations = {"TotalBeaten": graphlab.aggregate.SUM("NumberBeaten")})
teams_beaten.add_column(teams_beaten["TotalBeaten"]/ 9, "ExpectedWins") # 9 opponents
teams_beaten["TotalBeaten"] = teams_beaten["TotalBeaten"]/ weeks_elapsed # 
teams_beaten.rename({"TotalBeaten": "AvgBeatPerWeek"})
teams_beaten.sort("ExpectedWins", ascending = False)

# SFrames don't have round function?
# what if I wanted to do this to many columns?
# Sframe[[]]
teams_beaten["AvgBeatPerWeek"] = teams_beaten["AvgBeatPerWeek"].apply(lambda x: round(x,3))
teams_beaten["ExpectedWins"] = teams_beaten["ExpectedWins"].apply(lambda x: round(x,3))

# Include actual wins
# Include list of how many beaten in that week

teams_beaten.sort("ExpectedWins", ascending = False)

Team,AvgBeatPerWeek,ExpectedWins
Penguins,5.909,7.222
The Golden Yogis,5.273,6.444
Romosexual,5.0,6.111
theoutoftowner,4.909,6.0
dicksoutforharambe,4.364,5.333
Clippernation,4.091,5.0
NomoneyforRedzone,4.091,5.0
Sit4Anthems,4.0,4.889
Team 8,3.909,4.778
In TyGod we Trust,3.455,4.222


## Point Differentials

In [6]:
# Calculate Point Diff
matchups['Diff'] = abs(matchups['ScoreOne'] - matchups['ScoreTwo'])

In [7]:
matchups.head()

Week,TeamOne,ProjectedScoreOne,ScoreOne,Winner,ScoreTwo,ProjectedScoreTwo,TeamTwo
1,The Golden Yogis,107.69,102.2,<,149.46,120.48,Clippernation
1,In TyGod we Trust,109.61,90.46,<,108.44,109.49,Team 8
1,dicksoutforharambe,107.68,96.92,<,112.56,114.6,NomoneyforRedzone
1,Penguins,115.82,106.22,<,134.9,114.46,Romosexual
1,Sit4Anthems,106.49,73.1,<,118.16,98.82,theoutoftowner
2,The Golden Yogis,106.49,84.2,<,110.16,112.87,In TyGod we Trust
2,Clippernation,130.55,111.52,<,123.86,99.84,theoutoftowner
2,dicksoutforharambe,103.34,104.36,>,101.46,110.47,Team 8
2,Penguins,117.62,117.62,>,102.52,111.9,NomoneyforRedzone
2,Sit4Anthems,104.85,117.76,>,92.92,105.34,Romosexual


### Biggest Blowouts

In [8]:
biggest_blowouts = matchups.topk('Diff', 10)['Week', 'TeamOne', 'ScoreOne', 'Winner', 'ScoreTwo', 'TeamTwo', 'Diff']
biggest_blowouts
# format so all winner is on one side?
# is there a succinct way to do this?
# trouble because immutability of SFrames
# manage at list level
# or even more simple, change format when creating the csv
winner = list(biggest_blowouts["Winner"])
index = [x for x in range(0, len(winner)) if winner[x] == '<']
team_one = list(biggest_blowouts["TeamOne"])
team_two = list(biggest_blowouts["TeamTwo"])
score_one = list(biggest_blowouts["ScoreOne"])
score_two = list(biggest_blowouts["ScoreTwo"])

# else they would've pointed to the same list
temp_team = list(team_one)
temp_score = list(score_one)

for i in index:
    team_one[i] = team_two[i]
    score_one[i] = score_two[i]
    team_two[i] = temp_team[i]
    score_two[i] = temp_score[i]
    winner[i] = '>'

biggest_blowouts["TeamOne"] = graphlab.SArray(team_one)
biggest_blowouts["TeamTwo"] = graphlab.SArray(team_two)
biggest_blowouts["ScoreOne"] = graphlab.SArray(score_one)
biggest_blowouts["ScoreTwo"] = graphlab.SArray(score_two)
biggest_blowouts["Winner"] = graphlab.SArray(winner)
biggest_blowouts



Week,TeamOne,ScoreOne,Winner,ScoreTwo,TeamTwo,Diff
6,Penguins,159.6,>,70.36,In TyGod we Trust,89.24
3,In TyGod we Trust,128.88,>,57.64,theoutoftowner,71.24
5,Penguins,140.78,>,72.3,Clippernation,68.48
9,The Golden Yogis,157.28,>,91.98,theoutoftowner,65.3
3,Penguins,117.44,>,69.28,Team 8,48.16
1,Clippernation,149.46,>,102.2,The Golden Yogis,47.26
10,Penguins,126.42,>,79.4,Romosexual,47.02
3,Clippernation,116.58,>,70.9,dicksoutforharambe,45.68
1,theoutoftowner,118.16,>,73.1,Sit4Anthems,45.06
4,In TyGod we Trust,110.7,>,72.82,theoutoftowner,37.88


### Smallest Squeakers

In [9]:
squeakers = matchups.topk('Diff', 10, reverse = True)['Week', 'TeamOne', 'ScoreOne', 'Winner', 'ScoreTwo', 'TeamTwo', 'Diff']

winner = list(squeakers["Winner"])
index = [x for x in range(0, len(winner)) if winner[x] == '<']
team_one = list(squeakers["TeamOne"])
team_two = list(squeakers["TeamTwo"])
score_one = list(squeakers["ScoreOne"])
score_two = list(squeakers["ScoreTwo"])

# else they would've pointed to the same list
temp_team = list(team_one)
temp_score = list(score_one)

for i in index:
    team_one[i] = team_two[i]
    score_one[i] = score_two[i]
    team_two[i] = temp_team[i]
    score_two[i] = temp_score[i]
    winner[i] = '>'

squeakers["TeamOne"] = graphlab.SArray(team_one)
squeakers["TeamTwo"] = graphlab.SArray(team_two)
squeakers["ScoreOne"] = graphlab.SArray(score_one)
squeakers["ScoreTwo"] = graphlab.SArray(score_two)
squeakers["Winner"] = graphlab.SArray(winner)
squeakers


Week,TeamOne,ScoreOne,Winner,ScoreTwo,TeamTwo,Diff
8,Romosexual,100.0,>,99.32,In TyGod we Trust,0.68
9,dicksoutforharambe,109.16,>,108.24,Romosexual,0.92
2,dicksoutforharambe,104.36,>,101.46,Team 8,2.9
6,Romosexual,119.18,>,115.36,The Golden Yogis,3.82
7,Clippernation,116.02,>,110.52,Romosexual,5.5
10,theoutoftowner,129.88,>,124.14,Sit4Anthems,5.74
6,Sit4Anthems,100.94,>,94.98,Clippernation,5.96
7,NomoneyforRedzone,115.74,>,109.58,The Golden Yogis,6.16
11,The Golden Yogis,117.44,>,110.88,In TyGod we Trust,6.56
4,Romosexual,105.46,>,98.62,NomoneyforRedzone,6.84


## Avg Win/Loss Differential

In [10]:
# again, because of format, have to account for when team is on TeamOne or TeamTwo

t1_matchup_diff = matchups.groupby(key_columns = ["TeamOne", "Winner"], operations = {"SumDiff": graphlab.aggregate.SUM("Diff")})
t2_matchup_diff= matchups.groupby(key_columns = ["TeamTwo", "Winner"], operations = {"SumDiff": graphlab.aggregate.SUM("Diff")})

# flip Winner so we can match properly
winner_list = list(t2_matchup_diff["Winner"])
flip_list = ['>' if i == '<' else '<' for i in winner_list ]
t2_matchup_diff["Winner"] = graphlab.SArray(flip_list)


# have to technically match on name + winner, so we just combine them for now
t1_matchup_diff["TeamWinner"] = t1_matchup_diff[['TeamOne', 'Winner']].apply(lambda x: sorted(x.values()))

t2_matchup_diff["TeamWinner"] = t2_matchup_diff[['Winner', 'TeamTwo']].apply(lambda x: sorted(x.values()))

# oh ...there was a pack method for SFrames

# join
matchup_diff = t1_matchup_diff.join(t2_matchup_diff, on = "TeamWinner", how = "outer")


matchup_diff['SumDiff'] = matchup_diff[['SumDiff']].apply(lambda x: 0 if x['SumDiff'] is None else x['SumDiff'])
matchup_diff['SumDiff.1'] = matchup_diff[['SumDiff.1']].apply(lambda x: 0 if x['SumDiff.1'] is None else x['SumDiff.1'])

matchup_diff['TotalDiff'] = matchup_diff['SumDiff'] + matchup_diff['SumDiff.1']

matchup_diff

# okay really need to just fix this at csv level. 

matchup_wins_one = matchups.groupby(key_columns = ["TeamOne", "Winner"], operations = {"Count": graphlab.aggregate.COUNT})
matchup_wins_two = matchups.groupby(key_columns = ["TeamTwo", "Winner"], operations = {"Count": graphlab.aggregate.COUNT})
matchup_wins_one
winner_list = list(matchup_wins_two["Winner"])
flip_list = ['>' if i == '<' else '<' for i in winner_list ]
matchup_wins_two["Winner"] = graphlab.SArray(flip_list)

matchup_wins_one["TeamWinner"] = matchup_wins_one[['TeamOne', 'Winner']].apply(lambda x: sorted(x.values()))

matchup_wins_two["TeamWinner"] = matchup_wins_two[['Winner', 'TeamTwo']].apply(lambda x: sorted(x.values()))


matchup_wins = matchup_wins_one.join(matchup_wins_two, on = "TeamWinner", how = "outer")


matchup_wins['Count'] = matchup_wins[['Count']].apply(lambda x: 0 if x['Count'] is None else x['Count'])
matchup_wins['Count.1'] = matchup_wins[['Count.1']].apply(lambda x: 0 if x['Count.1'] is None else x['Count.1'])

matchup_wins['TotalCount'] = matchup_wins['Count'] + matchup_wins['Count.1']

matchup_final_diff = matchup_wins['TeamWinner', 'TotalCount'].join(matchup_diff['TeamWinner', 'TotalDiff'], \
                                                                   on = 'TeamWinner', how= 'inner')

matchup_final_diff['AvgDiff'] = matchup_final_diff['TotalDiff'] / matchup_final_diff['TotalCount']
matchup_final_diff['AvgDiff'] = matchup_final_diff['AvgDiff'].apply(lambda x: round(x,3))
matchup_final_diff = matchup_final_diff.unpack('TeamWinner')
matchup_final_diff['TeamWinner.0'] = matchup_final_diff['TeamWinner.0'].apply(lambda x: 'Wins' if x[0] == '>' else 'Losses')


matchup_final_diff.rename({'TeamWinner.0':'W/L', 'TeamWinner.1':'Team', 'TotalCount': 'Count', 'TotalDiff':'Diff'})
matchup_final_diff = matchup_final_diff['Team', 'W/L', 'Count', 'Diff','AvgDiff']
matchup_final_diff = matchup_final_diff.sort(['Team', 'W/L'], ascending = False)



### Sorted by Team

In [11]:
matchup_final_diff.head()

Team,W/L,Count,Diff,AvgDiff
theoutoftowner,Wins,7,161.96,23.137
theoutoftowner,Losses,4,192.68,48.17
dicksoutforharambe,Wins,5,61.52,12.304
dicksoutforharambe,Losses,6,163.84,27.307
The Golden Yogis,Wins,5,137.78,27.556
The Golden Yogis,Losses,6,124.44,20.74
Team 8,Wins,5,90.28,18.056
Team 8,Losses,6,140.84,23.473
Sit4Anthems,Wins,3,47.6,15.867
Sit4Anthems,Losses,8,197.54,24.692


In [12]:
matchup_final_diff.tail()

Team,W/L,Count,Diff,AvgDiff
Romosexual,Wins,7,119.72,17.103
Romosexual,Losses,4,78.28,19.57
Penguins,Wins,7,311.8,44.543
Penguins,Losses,4,104.68,26.17
NomoneyforRedzone,Wins,7,147.62,21.089
NomoneyforRedzone,Losses,4,50.38,12.595
In TyGod we Trust,Wins,5,163.88,32.776
In TyGod we Trust,Losses,6,154.38,25.73
Clippernation,Wins,4,115.8,28.95
Clippernation,Losses,7,150.9,21.557


### Sorted by Avg Diff over Wins and Losses

In [13]:
matchup_final_diff.sort(['W/L', 'AvgDiff'], ascending = False).head()

Team,W/L,Count,Diff,AvgDiff
Penguins,Wins,7,311.8,44.543
In TyGod we Trust,Wins,5,163.88,32.776
Clippernation,Wins,4,115.8,28.95
The Golden Yogis,Wins,5,137.78,27.556
theoutoftowner,Wins,7,161.96,23.137
NomoneyforRedzone,Wins,7,147.62,21.089
Team 8,Wins,5,90.28,18.056
Romosexual,Wins,7,119.72,17.103
Sit4Anthems,Wins,3,47.6,15.867
dicksoutforharambe,Wins,5,61.52,12.304


In [14]:
matchup_final_diff.sort(['W/L', 'AvgDiff'], ascending = False).tail()

Team,W/L,Count,Diff,AvgDiff
theoutoftowner,Losses,4,192.68,48.17
dicksoutforharambe,Losses,6,163.84,27.307
Penguins,Losses,4,104.68,26.17
In TyGod we Trust,Losses,6,154.38,25.73
Sit4Anthems,Losses,8,197.54,24.692
Team 8,Losses,6,140.84,23.473
Clippernation,Losses,7,150.9,21.557
The Golden Yogis,Losses,6,124.44,20.74
Romosexual,Losses,4,78.28,19.57
NomoneyforRedzone,Losses,4,50.38,12.595
